In [42]:
import numpy as np
import matplotlib.pyplot as plt
from parse import parse
import cv2

In [43]:
train_images, train_labels = parse('digitdata/trainingimages', 'digitdata/traininglabels', True)
validation_images, validation_labels = parse('digitdata/validationimages', 'digitdata/validationlabels', True)
test_images, test_labels = parse('digitdata/testimages', 'digitdata/testlabels', True)

In [44]:
base_rate_true = np.bincount(train_labels)/train_labels.shape[0]
print(base_rate_true)

[0.0958 0.1126 0.0976 0.0986 0.107  0.0868 0.1002 0.11   0.0924 0.099 ]


In [83]:
kernel = np.ones((3,3))
images = np.swapaxes(np.swapaxes(train_images.astype(np.float64), 0, 2), 0, 1)
dst = np.zeros_like(images)

if images.shape[2] > 512:
    for i in range(int(np.ceil(images.shape[2]/512))):
        dst[:, :, i*512:(i+1)*512] = cv2.filter2D(src = images[:, :, i*512:(i+1)*512], ddepth = -1, kernel = kernel, anchor = (0,0))
else:
    dst= cv2.filter2D(src = images, ddepth = -1, kernel = kernel, anchor = (0,0))

zones = dst[::3,::3,:]

    

    

In [92]:
def train_perceptron_numbers(passes, input, labels, zones):
    weights = np.zeros((81,10))
    bias = np.zeros(10)
    for k in range(passes):
        for i in range(len(input)):            
            b, index = 0, 0
            arr = zones[:,:,i].ravel()
            for j in range(len(weights[0])):
                a = np.dot(weights[:,j],arr) + bias[j]
                if a > b:
                    b = a
                    index = j
            if labels[i] != index:
                weights[:,index] -= arr
                bias[index] -= 1
                weights[:, labels[i]] += arr
                bias[labels[i]] += 1
    return weights, bias

   

In [93]:
vimages = np.swapaxes(np.swapaxes(validation_images.astype(np.float64), 0, 2), 0, 1)
vdst = np.zeros_like(vimages)

if vimages.shape[2] > 512:
    for i in range(int(np.ceil(vimages.shape[2]/512))):
        vdst[:, :, i*512:(i+1)*512] = cv2.filter2D(src = vimages[:, :, i*512:(i+1)*512], ddepth = -1, kernel = kernel, anchor = (0,0))
else:
    vdst= cv2.filter2D(src = vimages, ddepth = -1, kernel = kernel, anchor = (0,0))

vzones = vdst[::3,::3,:]

timages = np.swapaxes(np.swapaxes(test_images.astype(np.float64), 0, 2), 0, 1)
tdst = np.zeros_like(timages)

if timages.shape[2] > 512:
    for i in range(int(np.ceil(timages.shape[2]/512))):
        tdst[:, :, i*512:(i+1)*512] = cv2.filter2D(src = timages[:, :, i*512:(i+1)*512], ddepth = -1, kernel = kernel, anchor = (0,0))
else:
    tdst= cv2.filter2D(src = timages, ddepth = -1, kernel = kernel, anchor = (0,0))

tzones = tdst[::3,::3,:]


In [95]:
def run_perceptron_numbers(weights, input, labels, zones, bias):
    tally = 0
    for i in range(len(input)):
        b, index = 0, 0
        arr = zones[:,:,i].ravel()
        for j in range(len(weights[0])):
            a = np.dot(weights[:,j],arr) + bias[j]
            if a > b:
                b = a
                index = j
        if labels[i] == index:
            tally += 1

    print(tally/len(input))


In [97]:
w, b = train_perceptron_numbers(100, train_images, train_labels, zones)

run_perceptron_numbers(w, validation_images, validation_labels, vzones, b)

run_perceptron_numbers(w, test_images, test_labels, tzones, b)

0.852
0.842
